# Difference method + joint learning (fit_preset)

In this model, we will use what we called the Difference method. We first calculate the difference between the experimental band gap and the PBE band gap, and we then evaluate the error on this difference. We have three targets in the dataframe: the experimental and PBE band gaps, and the difference between them. We will apply joint learning on these columns.

In [1]:
def setup_threading():
    import os
    os.environ['OPENBLAS_NUM_THREADS'] = '1'
    os.environ['MKL_NUM_THREADS'] = '1'
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["TF_NUM_INTRAOP_THREADS"] = "1"
    os.environ["TF_NUM_INTEROP_THREADS"] = "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
setup_threading()

In [3]:
from modnet.preprocessing import MODData
from modnet.models.ensemble import EnsembleMODNetModel
import numpy as np
import os
import copy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [4]:
from sklearn.model_selection import KFold
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.reindex([ids])
    data.df_targets = data.df_targets.reindex([ids])
    data.df_structure = data.df_structure.reindex([ids])
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    #data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

def MD_append(md,lmd):
    md = copy.deepcopy(md)
    for m in lmd:
        md.df_structure.append(m.df_structure)
        md.df_targets.append(m.df_targets)
        md.df_featurized.append(m.df_featurized)
    return md

In [3]:
#from random import randint
#def train_val_split(data):

#    """Splits arrays or matrices into random train and validation subsets.
#
#   Parameter:
#      data: 'MODData' data which need to be splitted.
#    """
    #i = randint(0,9)
    #md_train, md_val = MDKsplit(data, n_splits=5, random_state=1)[0]
    #y_train = md_train.df_targets
    #y_val = md_val.df_targets

    #return md_train, md_val, y_train, y_val

In [4]:
#train_val_split(md_joint)

In [5]:
md_exp = MODData.load('exp_gap_all_mpid')
md_exp.df_targets.columns = ['exp_gap','mp_id']
md_pbe = MODData.load('pbe_gap.zip')
md_pbe.df_targets.columns = ['gap']
md_joint = MODData.load('exp_pbe_joint')


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766

2021-05-05 11:51:24,381 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f54f42e8700> object, created with modnet version <=0.1.7
2021-05-05 11:51:31,026 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f54f42eeac0> object, created with modnet version <=0.1.7
2021-05-05 11:51:31,050 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f54f42da760> object, created with modnet version <=0.1.7


In [6]:
md_joint.df_targets

,exp_gap,pbe_gap
mp-12699,0.00,0.0000
mp-559459,3.40,2.1704
mp-21162,0.00,0.0000
mp-1306,0.00,0.0000
mp-15252,0.53,0.1049
...,...,...
mp-556541,1.80,1.5135
mp-2602,1.04,0.0000
mp-30366,0.00,0.0000
mp-3718,1.45,0.2418


In [7]:
md_joint.df_targets['difference'] = md_joint.df_targets['exp_gap'] - md_joint.df_targets['pbe_gap']

In [8]:
md_joint.df_targets

,exp_gap,pbe_gap,difference
mp-12699,0.00,0.0000,0.0000
mp-559459,3.40,2.1704,1.2296
mp-21162,0.00,0.0000,0.0000
mp-1306,0.00,0.0000,0.0000
mp-15252,0.53,0.1049,0.4251
...,...,...,...
mp-556541,1.80,1.5135,0.2865
mp-2602,1.04,0.0000,1.0400
mp-30366,0.00,0.0000,0.0000
mp-3718,1.45,0.2418,1.2082


In [9]:
import pymatgen.ext.matproj

In [10]:
from pymatgen.ext.matproj import MPRester
import pandas as pd

with MPRester("v454AlrxL5Zg0cLAXW0") as mpr:
    index = md_joint.df_structure.index
    array = np.empty(shape=md_joint.df_structure.shape, dtype=pymatgen.core.structure.Structure)
    array = np.ndarray.flatten(array)
    for i in range(len(md_joint.df_structure)):
        array[i] = mpr.get_structure_by_material_id(index[i])
    md_joint.df_structure = pd.DataFrame(data=array, columns=["structure"])
    md_joint.df_structure = md_joint.df_structure.set_index(index)

In [11]:
#md_joint.save('exp_pbe_joint')

2021-05-05 11:58:44,955 - modnet - INFO - Data successfully saved as exp_pbe_joint!


In [5]:
md_joint = MODData.load('exp_pbe_joint')


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766

2021-05-06 09:25:42,115 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f571fb624f0> object, created with modnet version <=0.1.7


In [6]:
md_joint.df_structure

,structure
id,
mp-12699,"[[ 0.13616363 -2.60631371 1.50340165] Ca, [-1..."
mp-559459,"[[2.14049282 6.43464803 1.76199846] Cs, [-0.70..."
mp-21162,"[[0. 0. 0.] Sr, [0. 2.5259875 2.5259875..."
mp-1306,"[[3.259129 1.88166243 1.15971025] Th, [ 3.25..."
mp-15252,"[[0.95531657 1.042239 0.86229681] Cu, [2.803..."
...,...
mp-556541,"[[0. 6.49363948 5.82657217] In, [ 0. ..."
mp-2602,"[[3.023318 3.023318 3.023318] Nd, [0. 0. 0.] As]"
mp-30366,"[[4.51774558 1.37529211 4.32659015] Ca, [1.109..."


In [7]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(md_joint.df_targets['exp_gap'], md_joint.df_targets['pbe_gap'])

0.429783975697596

In [8]:
md_joint.df_targets['difference'].mad()

0.5494667780829975

In [9]:
k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
errors = np.ones((int(len(md_joint.df_targets)/k),5))
maes = np.ones(5)
uncertainties = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    #train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    fpath = 'train_joint_exp-pbe_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
       
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    # difference prediction
    model = EnsembleMODNetModel([[['difference']]],{'difference':1})
    model.fit_preset(train, verbose=0, n_jobs=20)
    
    pred, std = model.predict(test, return_unc=True)
    true = test.df_targets
    true = true.drop(columns=['exp_gap','pbe_gap'])
    error = pred-true
    error = error['difference'].drop(pred.index[((pred['difference']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    std = std['difference'].drop(std.index[((std['difference']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error).mean()
    uncertainty = np.abs(std).mean()
    print('mae')
    print(mae)
    print('uncertainty')
    print(uncertainty)
    maes[i] = mae
    uncertainties[i] = uncertainty
    

2021-05-06 09:25:45,765 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:45,781 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:45,794 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:45,810 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:45,823 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:45,838 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:45,849 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:45,865 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:45,875 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:45,891 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:46,323 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f571a486c40> object, created with modnet version 0.1.9
2021-05-06 09

  0%|          | 0/360 [00:00<?, ?it/s]

2021-05-06 09:32:51,746 - modnet - INFO - Preset #4 fitting finished, loss: 0.3741348087787628


  0%|          | 1/360 [07:00<41:55:13, 420.37s/it]

2021-05-06 09:34:22,178 - modnet - INFO - Preset #2 fitting finished, loss: 0.3645854890346527


...

2021-05-06 16:50:00,126 - modnet - INFO - Preset #55 fitting finished, loss: 0.3832109093666077


100%|█████████▉| 359/360 [1:27:05<00:42, 42.41s/it]

2021-05-06 16:51:13,534 - modnet - INFO - Preset #55 fitting finished, loss: 0.36173055768013


100%|██████████| 360/360 [1:28:18<00:00, 14.72s/it]


2021-05-06 16:51:21,978 - modnet - INFO - Preset #46 resulted in lowest validation loss with params {'train_data': <modnet.preprocessing.MODData object at 0x7f56c3060850>, 'targets': [[['difference']]], 'weights': {'difference': 1}, 'n_models': 5, 'num_classes': {'difference': 0}, 'n_feat': 256, 'num_neurons': [[256], [128], [32], [32]], 'lr': 0.005, 'batch_size': 64, 'epochs': 1000, 'loss': 'mae', 'act': 'elu', 'out_act': 'linear', 'callbacks': [<tensorflow.python.keras.callbacks.EarlyStopping object at 0x7f56fe117af0>], 'preset_id': 45, 'fold_id': 4, 'verbose': 0, 'val_data': <modnet.preprocessing.MODData object at 0x7f56c74d0f70>}
mae
0.18469601971686753
uncertainty
0.17916583


In [10]:
maes

array([0.34191074, 0.27340241, 0.3238936 , 0.27511023, 0.18469602])

In [11]:
maes.mean()

0.2798025996002459

In [12]:
uncertainties

array([0.22420746, 0.20271552, 0.22603963, 0.2025571 , 0.17916583])

In [13]:
uncertainties.mean()

0.2069371074438095

#### Conclusion

Relative percentage change of the error compared to MODNet_exp_GA reference: 19%